In [1]:
import sys, os
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
from astropy.io import ascii
%matplotlib inline

In [2]:
import lsst.sims.maf.db as db
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.metricBundles as mb
from lsst.sims.utils import Site
from lsst.sims.utils import calcLmstLast
from lsst.ts.astrosky.model import AstronomicalSkyModel

ModuleNotFoundError: No module named 'lsst.ts'

Testing algorithm to compute observational season start/end dates for fields. In principle a season starts when the field is rising at maximum airmass at the end of the night and ends when the field is setting at maximum airmass at the start of the night.

In [3]:
# Some definitions
max_airmass = 1.5
nvisits = { 'y' : 210,
            'z' : 210,
            'i' : 240,
            'r' : 240,
            'g' : 105,
            'u' : 75
          }

In [4]:
o = db.OpsimDatabase(os.path.expanduser('~/Data/db/colossus_2296.db'))

In [5]:
# Query fields from WFD
sql_query = '''select * from Field join ProposalField on 
Field.fieldId == ProposalField.Field_fieldId 
where ProposalField.Proposal_propId == 3
'''
d = o.query_arbitrary(sql_query)

In [11]:
print('Fields in WFD survey: %i' % len(d))

Fields in WFD survey: 2293


In [10]:
# datermine date start from the first observation
date_start = o.query_columns('SummaryAllProps', colnames=['observationStartMJD'], 
                    sqlconstraint='night == 1 and observationId == 1')[0][0]
print('First observation @ %.4f' % date_start)

First observation @ 59853.0168


In [12]:
site = Site('LSST')